In [1]:
import pandas as pd
import cytoolz as tlz
from itertools import chain


In [5]:
fin_df = pd.read_excel('./global_findex_database.xlsx', sheet_name = 'Series_Table')

In [154]:
series = fin_df['Series']
series.head()

0       account.t.d
1     account.t.d.1
2    account.t.d.10
3    account.t.d.11
4     account.t.d.2
Name: Series, dtype: object

In [124]:
main_metrics = []
series_len = len(series)

for i, curr_metric in enumerate(series, 1):
    if len(main_metrics) == 0:
        main_metrics.append((curr_metric, i))
        continue
        
    metric, _ = tlz.last(main_metrics)
    is_diff_metric = metric not in curr_metric
    is_series_end = i == series_len 
    
    if is_diff_metric:
        recent_metric, index = main_metrics.pop()
        main_metrics.append((recent_metric, (i-1) - index))
        main_metrics.append((curr_metric, i))
        
    if is_series_end:
        recent_metric, index = main_metrics.pop()
        main_metrics.append((recent_metric, series_len - index))

In [166]:
len(main_metrics)

135

In [167]:
main_series = pd.Series(
    chain.from_iterable(
        [ [metric] + ['-'] * n for metric, n in main_metrics]
    ), 
    name='Main Series'
)

In [164]:
metric_df = pd.concat([main_series, series], axis=1)
metric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 2 columns):
Main Series    776 non-null object
Series         776 non-null object
dtypes: object(2)
memory usage: 12.2+ KB


In [ ]:
del fin_df['Series']

In [163]:
fin_metric_df = pd.concat([metric_df, fin_df], axis=1)

fin_metric_df.head()

fin_metric_df.to_csv('collapsed.csv')